In [1]:
import cv2
import pygame
import numpy as np
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

pygame 2.6.0 (SDL 2.28.4, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
verts = (
  (1, -1, -1),
  (1, 1, -1),
  (-1, 1, -1),
  (-1, -1, -1),
  (1, -1, 1),
  (1, 1, 1),
  (-1, -1, 1),
  (-1, 1, 1))

edges = (
  (0,1),(0,3),(0,4),
  (2,1),(2,3),(2,7),
  (6,3),(6,4),(6,7),
  (5,1),(5,4),(5,7))

surfaces = (
  (0,1,2,3),
  (3,2,7,6),
  (6,7,5,4),
  (4,5,1,0),
  (1,5,7,2),
  (4,0,3,6))

colors = (
  (1,0,0),
  (0,1,0),
  (0,0,1),
  (1,1,0),
  (0,1,1),
  (1,0,1))    

In [3]:
frame, gray = 0, 0

displayAxis = False
displayInformation = True

button = False
loop = True
clock, frameCount = 0, 0

faceType = False
ym = 5.4                                    # display image y margin
rotX, rotY = 0, 0
scale, zScale = 2., 1.                      # opengl display scale, z axis scale

msg = "..."
font, fontSize = 0, 18
fRate, size = 0.1, 5
mouseX, mouseY, pmouseX, pmouseY = 0, 0, 0, 0

objWidth, objHeight = 1, 1                # face object width, height
minDepth, meanDepth, maxDepth = -1, 0, 1     # face object min, max, mean depth 

eye = (0., 0., 10.)
center = (0., 0., 0.)
fps, delay = 30, 1 # 33


In [4]:

def myInit():
    global winWidth, winHeight
    global font, fontSize, loop, frameCount, clock
    loop = True
    frameCount = 0

    pygame.init()
    clock = pygame.time.Clock()

    pygame.display.set_caption('pygameOpenGL introduction lbg@dongseo.ac.kr')
    screen = pygame.display.set_mode((winWidth, winHeight), DOUBLEBUF|OPENGL) # FULLSCREEN
    font = pygame.font.SysFont("courier new", fontSize, bold=True) #'arial'


def myReshape():
    global winWidth, winHeight
    glClearColor (.75, .75, .75, 0.0)
    glViewport(0, 0, winWidth, winHeight)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(60., winWidth/winHeight, .1, 10000.)
    glMatrixMode(GL_MODELVIEW)
    glEnable(GL_DEPTH_TEST)
    

def drawText(x, y, text, gray): 
    textSurface = font.render(text, True, (255, 255, 255, 255), (gray, gray, gray, 255))
    textData = pygame.image.tostring(textSurface, "RGBA", True)
    glWindowPos2d(x, y)
    glDrawPixels(textSurface.get_width(), textSurface.get_height(), GL_RGBA, GL_UNSIGNED_BYTE, textData)


def myAxis():
    global objWidth, objHeight
    global minDepth, maxDepth, meanDepth

    glPushMatrix()
    glTranslatef(-objWidth/2, -objHeight/2, 0)

    glColor3f(0., 1., 0.)
    glLineWidth(2)
    glBegin(GL_LINE_LOOP)
    glVertex3f(0, 0, meanDepth)
    glVertex3f(objWidth, 0, meanDepth)
    glVertex3f(objWidth, objHeight, meanDepth)
    glVertex3f(0, objHeight, meanDepth)
    glEnd()
    glBegin(GL_LINE_LOOP)
    glVertex3f(0, 0, minDepth)
    glVertex3f(objWidth, 0, minDepth)
    glVertex3f(objWidth, objHeight, minDepth)
    glVertex3f(0, objHeight, minDepth)
    glEnd()
    glBegin(GL_LINE_LOOP)
    glVertex3f(0, 0, maxDepth)
    glVertex3f(objWidth, 0, maxDepth)
    glVertex3f(objWidth, objHeight, maxDepth)
    glVertex3f(0, objHeight, maxDepth)
    glEnd()

    glColor3f(1., 0., 1.) 
    glBegin(GL_LINES)
    glVertex3f(0, 0, minDepth)
    glVertex3f(0, 0, maxDepth)
    glVertex3f(objWidth, 0, minDepth)
    glVertex3f(objWidth, 0, maxDepth)
    glVertex3f(objWidth, objHeight, minDepth)
    glVertex3f(objWidth, objHeight, maxDepth)
    glVertex3f(0, objHeight, minDepth)
    glVertex3f(0, objHeight, maxDepth)
    glEnd()
    glPopMatrix()

    
def myFace():
    glLineWidth(1)
    if faceType:
        glBegin(GL_QUADS)
        i = 0
        for surface in surfaces:
            glColor3fv(colors[i%len(colors)])
            for vertex in surface:
                glVertex3fv(verts[vertex])
            i += 1    
        glEnd()
    else: 
        glBegin(GL_LINES)
        i = 0
        for edge in edges:
            glColor3fv(colors[i%len(colors)])
            for vertex in edge:
                glVertex3fv(verts[vertex])
            i += 1    
        glEnd()

def myMessage():
    global winWidth, winHeight, fontSize, clock   
    global displayInformation, displayAxis
    global minDepth, meanDepth, maxDepth

    w, h = 5.8, 1.4
    glPushMatrix()
    glTranslatef(-8.2, ym-h, 0)
    glColor3f(.3, .3, .3)
    glBegin(GL_QUADS)
    glVertex3f(0, 0, 0)
    glVertex3f(w, 0, 0)
    glVertex3f(w, h, 0)
    glVertex3f(0, h, 0)
    glEnd()
    glPopMatrix()
    
    xmargin = 40
    ymargin = 40
    gray = 76
    drawText(xmargin, winHeight-ymargin-fontSize, "inform(i)"+str(displayInformation)+" scale(z/x)"+'{:.2f}'.format(scale), gray)
    drawText(xmargin, winHeight-ymargin-2*fontSize, "faceType(f)"+str(faceType) + " reset rotation(d)", gray)
    drawText(xmargin, winHeight-ymargin-3*fontSize, "axis(a)"+str(displayAxis)+" fps:" + str(int(clock.get_fps()))+"-"+str(frameCount), gray)
    drawText(xmargin, winHeight-ymargin-4*fontSize, msg, gray)
    

def myDisplay():    
    global eye, center, delay, rotX, rotY, scale
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT) 
    glLoadIdentity()
    gluLookAt(eye[0], eye[1], eye[2], center[0], center[1], center[2], 0, 1, 0)

    if displayInformation:
        myMessage()
    
    glRotatef(rotX, 1, 0, 0)
    glRotatef(rotY, 0, 1, 0)
    glScalef(scale, scale, scale)
    
    myFace()
    
    if displayAxis:
        myAxis()

    pygame.display.flip()
    pygame.time.wait(delay)


def myQuit():
    pygame.quit()
    cv2.destroyAllWindows()
    

In [5]:
def myEvent():
    global loop, faceType, rotX, rotY, scale, msg
    global mouseX, mouseY, pmouseX, pmouseY, button
    global displayInformation, displayAxis, frameCount
    frameCount+=1

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            loop = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                loop = False
                msg = "ESC stop..."
            elif event.key == pygame.K_i:
                displayInformation = not displayInformation
                msg = "i key information : " + str(displayInformation)
            elif event.key == pygame.K_a:
                displayAxis = not displayAxis
                msg = "a key displayAxis : " + str(displayAxis)
            elif event.key == pygame.K_d:
                rotX = rotY = 0
                msg = "d key reset rot"
            elif event.key == pygame.K_f:
                faceType = not faceType
                msg = "f key faceType"
            elif event.key == pygame.K_z:
                scale *= 1.075
                msg = "z key scale : " + '{:.2f}'.format(scale)
            elif event.key == pygame.K_x:
                scale /= 1.075
                msg = "x key scale : " + '{:.2f}'.format(scale)
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mouseX, mouseY = pygame.mouse.get_pos()
            pmouseX, pmouseY, button = mouseX, mouseY, True
            msg = "mouse down (" + str(mouseX) + "," + str(mouseY)+ ") " + str(button)
        elif event.type == pygame.MOUSEMOTION:
            mouseX, mouseY = pygame.mouse.get_pos()
            if button == True:
                rotX -= (pmouseY - mouseY)*fRate
                rotY -= (pmouseX - mouseX)*fRate
                pmouseX, pmouseY = mouseX, mouseY
        elif event.type == pygame.MOUSEBUTTONUP:
            mouseX, mouseY = pygame.mouse.get_pos()
            button = False
            msg = "mouse up (" + str(mouseX) + "," + str(mouseY)+ ") " + str(button)   
       

In [6]:
fps = 30                                    
winWidth, winHeight = 1200, 800
myInit()
myReshape()
while(loop):
    myDisplay()
    myEvent()    
    clock.tick(fps)
myQuit()